# 02 - Memory Capture & Distill: The 4 Memory Types

This notebook dives deep into **how memories are created, stored, and organised** across 4 memory types.

## The 4 Memory Types

| Type | What It Stores | How It's Created | Backend |
|------|---------------|------------------|---------|
| **Short-Term** | Recent conversation turns | Every `.chat()` call | Supabase |
| **Semantic (LT)** | Distilled facts & preferences | LLM extraction from conversation | Supabase pgvector |
| **Episodic** | Full conversation sessions | End-of-session snapshot | Supabase pgvector |
| **Procedural** | Step-by-step workflows | Pre-loaded by admin | Supabase pgvector |

## What You'll Learn
1. How **short-term memory** works as a ring buffer
2. How the **distiller** extracts long-term facts from conversation turns
3. How **episodic memory** preserves full conversation context
4. How **procedural memory** guides the agent through multi-step tasks
5. How all 4 memory types work together in the agent pipeline

## Prerequisites
- Run Notebook 01 first (or `make seed-crm-xl` + `make seed-procedures`)

In [1]:
#  Setup ─
import sys, os, re, time, json, random
sys.path.insert(0, "../src")

from dotenv import load_dotenv
load_dotenv()

# Configure loguru
from infrastructure.log import setup_logging
from loguru import logger
setup_logging("INFO", for_notebook=True)

import pandas as pd
from datetime import datetime
from sqlalchemy.orm import sessionmaker

from memory import (
    ConversationTurn,
    ShortTermMemoryStore,
    LongTermMemoryStore,
    EpisodicMemoryStore,
    ProceduralMemoryStore,
    MemoryDistiller,
    MemoryRecaller,
    create_episode_from_turns,
)
from services.crm_service import get_crm_client
from infrastructure.db import create_tables, get_sql_engine
from infrastructure.db.crm_models import Patient, Booking, Doctor, Location, Specialty
from infrastructure.llm import get_chat_llm, get_default_embeddings
from infrastructure.config import ST_MAX_TURNS, ST_TTL_SECONDS

# Initialise
create_tables()
embedder = get_default_embeddings()
llm = get_chat_llm()
crm = get_crm_client()

st_store = ShortTermMemoryStore()
lt_store = LongTermMemoryStore(embedder)
distiller = MemoryDistiller(llm, lt_store)

logger.success("Short-term memory : Supabase (st_turns)")
logger.success("Long-term memory  : Supabase pgvector")
logger.success("CRM client        : ready")
logger.success(f"LLM               : {llm.model_name if hasattr(llm, 'model_name') else 'unknown'}")

/Users/machinelearningzuu/Dropbox/Zuu Crew/Courses/🚧 AI Engineer Essentials/Codes/Agentic Memory Design/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ℹ️ ✓ Supabase SQL engine created
ℹ️ ✅ Supabase connection test: SUCCESS
ℹ️ ✅ pgvector extension: INSTALLED
ℹ️ ✓ Schema validation passed: vector(1536)
ℹ️ ✓ Database tables created/verified
✅ Short-term memory : Supabase (st_turns)
✅ Long-term memory  : Supabase pgvector
✅ CRM client        : ready
✅ LLM               : google/gemini-2.5-flash


---

## Part 1 · Short-Term Memory - Conversation Ring Buffer

Short-term memory stores the **most recent N conversation turns** per user/session.

- **Backend**: Supabase (`st_turns` table)
- **Capacity**: Configurable ring buffer (default 10 turns)
- **TTL**: Turns expire after a configurable timeout (default 24h)
- **Purpose**: Provides conversational continuity within a session

In [ ]:
#  Intelligent phone extraction ─
# Users type numbers in many formats - this handles them all.

def extract_phone(text: str) -> str:
    """Extract and normalise a Sri Lankan phone number from free-form text."""
    match = re.search(r"\+?[\d][\d\s\-\.\(\)]{7,18}[\d]", text)
    if not match:
        raise ValueError(" No phone number found in the message!")
    raw = re.sub(r"\D", "", match.group())
    if raw.startswith("0") and len(raw) == 10:
        raw = "94" + raw[1:]           # local → international
    elif len(raw) == 9 and not raw.startswith("94"):
        raw = "94" + raw               # bare subscriber number
    logger.info(f"   Normalised → {raw}")
    return raw


#  Identify the user from a chat message 
# In a real system the phone comes through the chat interface.
# Try any format: "0781030736", "+94-781-030-736", "+94 78 10 30 736"

greeting = "Hello! I'm Anushka, here for a follow-up. My mobile is 0781030736."

user_id = extract_phone(greeting)
logger.success(f" Extracted phone → user_id = {user_id}")

session_id = "nb02-demo"

#  Look up patient in CRM and display ─
patient = crm.get_patient_by_user_id(user_id)
name = patient["full_name"].split()[0] if patient else "TestUser"

if patient:
    df = pd.DataFrame([{
        "Field": k, "Value": v
    } for k, v in {
        "Patient ID": patient["patient_id"],
        "Full Name": patient["full_name"],
        "Phone": patient.get("phone", "-"),
        "Email": patient.get("email", "-"),
        "DOB": patient.get("dob", "-"),
        "Gender": {"M": "Male", "F": "Female", "X": "Other"}.get(
            patient.get("gender", ""), patient.get("gender", "-")
        ),
    }.items()])
    print(" Patient Record  (Supabase → patients table)")
    display(df.style.hide(axis="index"))

    # Show bookings
    _session = sessionmaker(bind=get_sql_engine())()
    try:
        pat_obj = _session.query(Patient).filter(
            Patient.external_user_id == user_id
        ).first()
        if pat_obj:
            rows_raw = (
                _session.query(Booking, Doctor, Location)
                .join(Doctor, Booking.doctor_id == Doctor.doctor_id)
                .join(Location, Booking.location_id == Location.location_id)
                .filter(Booking.patient_id == pat_obj.patient_id)
                .order_by(Booking.start_at.desc())
                .limit(10)
                .all()
            )
            if rows_raw:
                bk_rows = []
                for bk, doc, loc in rows_raw:
                    spec = (
                        _session.get(Specialty, doc.specialty_id)
                        if doc.specialty_id else None
                    )
                    bk_rows.append({
                        "Date": datetime.fromtimestamp(bk.start_at).strftime("%Y-%m-%d %H:%M"),
                        "Doctor": f"Dr. {doc.full_name}",
                        "Specialty": spec.name if spec else "-",
                        "Location": loc.name,
                        "Status": bk.status,
                        "Reason": bk.reason or "-",
                    })
                print(f"\n Bookings  ({len(bk_rows)} records)")
                display(pd.DataFrame(bk_rows))
    finally:
        _session.close()
else:
    logger.warning("  No patient found - using defaults")

ℹ️    Normalised → 94781030736
✅ 📱 Extracted phone → user_id = 94781030736
📋 Patient Record  (Supabase → patients table)


Field,Value
Patient ID,12692f5d-1630-4ecd-bf7e-bcfd08260b73
Full Name,Anushka Perera
Phone,+94781030736
Email,anushka.perera@gmail.com
DOB,1985-03-15
Gender,Female



📅 Bookings  (7 records)


,Date,Doctor,Specialty,Location,Status,Reason
0,2025-11-11 20:00,Dr. Dr. Tharindu Sivasubramaniam,Cardiology,Nawaloka City OPD,PENDING,Preoperative cardiac risk assessment
1,2025-11-10 16:00,Dr. Dr. Sita Thirunavukarasu,Cardiology,Nawaloka City OPD,PENDING,Cardiac stress test referral
2,2025-11-07 19:30,Dr. Dr. Chamara Dissanayake,Pediatrics,Heart Care Clinic,CONFIRMED,Immunization update and review
3,2025-11-06 17:00,Dr. Dr. Kusal Fernando,Dermatology,Heart Care Clinic,CONFIRMED,Actinic keratosis monitoring and intervention
4,2025-11-04 18:30,Dr. Dr. Anushka Selvaraj,Orthopedics,Nawaloka City OPD,PENDING,Surgical consultation for meniscus tear
5,2025-11-03 20:30,Dr. Dr. Ramesh Gunasekara,Pediatrics,Nawaloka City OPD,PENDING,Nutritional counseling for obesity
6,2025-11-03 16:00,Dr. Dr. Tharika Suresh,Pathology,Central Lab,CONFIRMED,Neuropathic pain evaluation and management


In [ ]:
#  Build a realistic conversation ─
# Tailored to Anushka Perera's CRM profile:
#   - Has cardiology bookings (cardiac risk assessment, stress test)
#   - Has orthopedics booking (meniscus tear consultation)
#   - DOB: 1985-03-15, Female

conversations = [
    ("user",      f"Hi, I'm {name}. I have a cardiac stress test coming up."),
    ("assistant", f"Hello {name}! I can see that in your records. How can I help you prepare?"),
    ("user",      f"From now on, remember that I take atenolol 50mg every morning for blood pressure."),
    ("assistant", f"Got it! I'll remember your atenolol 50mg daily medication schedule."),
    ("user",      f"I'm allergic to penicillin, please always remember this."),
    ("assistant", f"Important! I've noted your penicillin allergy. This is critical information."),
    ("user",      f"Also remind me that I have a meniscus tear follow-up with orthopedics."),
    ("assistant", f"Noted! I'll remember your orthopedics follow-up for the meniscus tear."),
]

# Store each turn in short-term memory
print(f" Storing {len(conversations)} turns in short-term memory...\n")
for role, content in conversations:
    turn = ConversationTurn(
        user_id=user_id,
        session_id=session_id,
        role=role,
        content=content,
        ts=time.time(),
    )
    st_store.append(turn, ST_MAX_TURNS, ST_TTL_SECONDS)
    emoji = "👤" if role == "user" else "🤖"
    print(f"  {emoji} [{role:9s}]: {content}")
    time.sleep(0.1)  # small delay for timestamp ordering

logger.success(f"\n {len(conversations)} turns stored")

💾 Storing 8 turns in short-term memory...

  👤 [user     ]: Hi, I'm Anushka. I have a cardiac stress test coming up.
  🤖 [assistant]: Hello Anushka! I can see that in your records. How can I help you prepare?
  👤 [user     ]: From now on, remember that I take atenolol 50mg every morning for blood pressure.
  🤖 [assistant]: Got it! I'll remember your atenolol 50mg daily medication schedule.
  👤 [user     ]: I'm allergic to penicillin, please always remember this.
  🤖 [assistant]: Important! I've noted your penicillin allergy. This is critical information.
  👤 [user     ]: Also remind me that I have a meniscus tear follow-up with orthopedics.
  🤖 [assistant]: Noted! I'll remember your orthopedics follow-up for the meniscus tear.
✅ 
✅ 8 turns stored


In [ ]:
#  Retrieve from short-term memory 
recent_turns = st_store.recent(user_id, session_id, k=10)

print(f" Retrieved {len(recent_turns)} recent turns from ST memory:\n")
for i, turn in enumerate(recent_turns, 1):
    emoji = "👤" if turn.role == "user" else "🤖"
    age = f"{time.time() - turn.ts:.0f}s ago"
    print(f"  {i}. {emoji} [{turn.role:9s}] ({age}): {turn.content}")

📤 Retrieved 10 recent turns from ST memory:

  1. 👤 [user     ] (90524s ago): Also remind me that I have a meniscus tear follow-up with orthopedics.
  2. 🤖 [assistant] (90522s ago): Noted! I'll remember your orthopedics follow-up for the meniscus tear.
  3. 👤 [user     ] (13s ago): Hi, I'm Anushka. I have a cardiac stress test coming up.
  4. 🤖 [assistant] (12s ago): Hello Anushka! I can see that in your records. How can I help you prepare?
  5. 👤 [user     ] (10s ago): From now on, remember that I take atenolol 50mg every morning for blood pressure.
  6. 🤖 [assistant] (8s ago): Got it! I'll remember your atenolol 50mg daily medication schedule.
  7. 👤 [user     ] (7s ago): I'm allergic to penicillin, please always remember this.
  8. 🤖 [assistant] (5s ago): Important! I've noted your penicillin allergy. This is critical information.
  9. 👤 [user     ] (4s ago): Also remind me that I have a meniscus tear follow-up with orthopedics.
  10. 🤖 [assistant] (2s ago): Noted! I'll remember you

---

## Part 2 · Semantic Memory - Distillation (LT Facts)

The **distiller** uses an LLM to extract important facts from conversation turns.

**Trigger policy** - distillation fires when:
- Turn count ≥ 5, OR
- Conversation contains keywords: `"remember"`, `"from now on"`, `"remind me"`, `"always"`, `"never"`

**Process**: Conversation → LLM → JSON array of facts → scored → deduped → stored in pgvector

In [5]:
#  Check distillation policy 
should = distiller.should_distill(recent_turns)

print(f"Should distill? {should}")
print(f"\nTrigger analysis:")
print(f"  Turn count    : {len(recent_turns)} (threshold: 5)")
print(f"  'remember'    : {any('remember' in t.content.lower() for t in recent_turns)}")
print(f"  'from now on' : {any('from now on' in t.content.lower() for t in recent_turns)}")
print(f"  'remind me'   : {any('remind me' in t.content.lower() for t in recent_turns)}")
print(f"  'always'      : {any('always' in t.content.lower() for t in recent_turns)}")

Should distill? True

Trigger analysis:
  Turn count    : 10 (threshold: 5)
  'remember'    : True
  'from now on' : True
  'remind me'   : True
  'always'      : True


In [ ]:
#  Run distillation ─
print("  Distilling facts from conversation...")
print("   (LLM call - extracting structured facts)\n")

facts = distiller.distill(user_id, recent_turns)

logger.success(f" Extracted {len(facts)} long-term facts:\n")
for i, fact in enumerate(facts, 1):
    print(f"  {i}. {fact.text}")
    print(f"     Score: {fact.score:.3f}  |  Tags: {', '.join(fact.tags)}")
    print(f"     ID: {fact.id}")
    print()

🧠 Distilling facts from conversation...
   (LLM call - extracting structured facts)

ℹ️ LangFuse client initialised (host=https://us.cloud.langfuse.com)
ℹ️ Upserted 4 facts to LT memory (0 new, 4 merged)
ℹ️ Distilled 4 facts for user 94781030736
✅ ✅ Extracted 4 long-term facts:

  1. Anushka has a meniscus tear follow-up with orthopedics.
     Score: 0.587  |  Tags: appointment, orthopedics, meniscus_tear, follow_up
     ID: af6e6c18-dd0a-4cec-876a-6e14a099b7f8

  2. The user's name is Anushka.
     Score: 0.587  |  Tags: preference, personal_info
     ID: a4943455-87df-4b03-8f76-b71f254b9bd7

  3. Anushka takes Atenolol 50mg every morning for blood pressure.
     Score: 0.587  |  Tags: medication, blood_pressure, schedule, prescription
     ID: 93e244d1-5eb1-42ea-abdc-735adb8f7188

  4. Anushka is allergic to penicillin.
     Score: 0.587  |  Tags: allergy, penicillin, allergic_reaction
     ID: 46396b75-c7f6-4482-b541-8a94cfbb04cd



In [ ]:
#  Verify facts are queryable via semantic search ─
test_queries = ["blood pressure medication", "drug allergies", "orthopedics follow-up"]

logger.info(" Querying LT memory (semantic search via pgvector):\n")
for query in test_queries:
    results = lt_store.query(user_id, query, k=3, threshold=0.3)
    print(f"  Query: '{query}'  →  {len(results)} fact(s)")
    for fact in results:
        print(f"    ↳ {fact.text}")
    print()

ℹ️ 🔍 Querying LT memory (semantic search via pgvector):

ℹ️ Retrieved 3 facts from LT memory for user 94781030736
  Query: 'blood pressure medication'  →  3 fact(s)
    ↳ Remind Anushka to take Atenolol 50mg daily for blood pressure
    ↳ Anushka takes atenolol 50mg every morning for blood pressure
    ↳ Anushka takes atenolol 50mg every morning for blood pressure

ℹ️ Retrieved 3 facts from LT memory for user 94781030736
  Query: 'drug allergies'  →  3 fact(s)
    ↳ Always remember Anushka's allergy to penicillin
    ↳ Remind Anushka of penicillin allergy always
    ↳ Anushka is allergic to penicillin and this information should always be remembered.

ℹ️ Retrieved 3 facts from LT memory for user 94781030736
  Query: 'orthopedics follow-up'  →  3 fact(s)
    ↳ Remind Anushka about her meniscus tear follow-up with orthopedics
    ↳ Anushka has a meniscus tear follow-up with orthopedics.
    ↳ Inform doctor or nurse about current medications during appointments



---

## Part 3 · Episodic Memory - Full Conversation Snapshots

Episodic memory stores **complete conversation sessions** - not just facts.

**Semantic vs. Episodic:**
- **Semantic**: `"User takes blood pressure medication at 8am"` (extracted fact)
- **Episodic**: The full 8-turn conversation where this was discussed, with LLM summary

This enables queries like *"What did we discuss last week?"*

In [ ]:
#  Create and store an episode ─
episodic_store = EpisodicMemoryStore(embedder)

# create_episode_from_turns uses LLM to generate a summary
episode = create_episode_from_turns(
    user_id=user_id,
    session_id=session_id,
    turns=recent_turns,
    llm=llm,   # LLM generates an intelligent summary
)

print(f" Episode Created:")
print(f"  ID       : {episode.id}")
print(f"  Session  : {episode.session_id}")
print(f"  Turns    : {episode.turn_count}")
print(f"  Duration : {episode.end_at - episode.start_at:.1f}s")
print(f"  Topics   : {', '.join(episode.topic_tags)}")
print(f"  Summary  : {episode.summary}")

# Store it
print("\n Storing episode in episodic memory...")
episodic_store.store_episode(episode)
logger.success(" Episode stored!")

ℹ️ ✓ Episodic memory store initialized (Supabase/pgvector)
📦 Episode Created:
  ID       : 870aa44c-cfa6-427f-a691-0377dfe4be09
  Session  : nb02-demo
  Turns    : 10
  Duration : 90521.9s
  Topics   : medication, allergy
  Summary  : The user is providing the assistant with important medical information, including a meniscus tear follow-up, a cardiac stress test, a daily medication (atenolol), and a penicillin allergy, expecting the assistant to remember these details.

💾 Storing episode in episodic memory...
ℹ️ ✓ Stored episode 870aa44c-cfa6-427f-a691-0377dfe4be09 with 10 turns
✅ ✅ Episode stored!


In [ ]:
#  Query episodic memory (semantic search on summaries) ─
queries = ["medication discussion", "allergies and safety", "appointment history"]

logger.info(" Querying episodic memory:\n")
for q in queries:
    episodes = episodic_store.query_episodes(user_id, q, k=2, threshold=0.3)
    print(f"  Query: '{q}'  →  {len(episodes)} episode(s)")
    for ep in episodes:
        print(f"     {ep.session_id} | {ep.turn_count} turns | Topics: {', '.join(ep.topic_tags)}")
        print(f"       Summary: {ep.summary[:120]}...")
    print()

ℹ️ 🔍 Querying episodic memory:

ℹ️ Retrieved 2 episodes for user 94781030736
  Query: 'medication discussion'  →  2 episode(s)
    📦 nb02-demo | 10 turns | Topics: medication, allergy
       Summary: The user is providing the assistant with important medical information, including a meniscus tear follow-up, a cardiac s...
    📦 nb02-demo | 10 turns | Topics: medication, allergy
       Summary: The user, Anushka, discussed her upcoming cardiac stress test and requested reminders for her meniscus tear follow-up wi...

ℹ️ Retrieved 2 episodes for user 94781030736
  Query: 'allergies and safety'  →  2 episode(s)
    📦 nb02-demo | 10 turns | Topics: medication, allergy
       Summary: The user, Anushka, discussed her upcoming cardiac stress test and requested reminders for her meniscus tear follow-up wi...
    📦 nb02-demo | 10 turns | Topics: medication, allergy
       Summary: The user is providing the assistant with important medical information, including a meniscus tear follow-up, a car

In [ ]:
#  Retrieve full episode (all turns) 
retrieved = episodic_store.get_episode_by_session(user_id, session_id)

if retrieved:
    print(f" Full Episode: {retrieved.session_id}")
    print(f"   Summary: {retrieved.summary}\n")
    print("=" * 60)
    for i, turn in enumerate(retrieved.turns, 1):
        emoji = "👤" if turn.role == "user" else "🤖"
        print(f"  {i}. {emoji} [{turn.role.upper()}]: {turn.content}")
    print("=" * 60)
    logger.success("\n Episodic memory preserves the FULL conversation context.")
else:
    logger.error(" Episode not found")

📖 Full Episode: nb02-demo
   Summary: The user is providing the assistant with important medical information, including a meniscus tear follow-up, a cardiac stress test, a daily medication (atenolol), and a penicillin allergy, expecting the assistant to remember these details.

  1. 👤 [USER]: Also remind me that I have a meniscus tear follow-up with orthopedics.
  2. 🤖 [ASSISTANT]: Noted! I'll remember your orthopedics follow-up for the meniscus tear.
  3. 👤 [USER]: Hi, I'm Anushka. I have a cardiac stress test coming up.
  4. 🤖 [ASSISTANT]: Hello Anushka! I can see that in your records. How can I help you prepare?
  5. 👤 [USER]: From now on, remember that I take atenolol 50mg every morning for blood pressure.
  6. 🤖 [ASSISTANT]: Got it! I'll remember your atenolol 50mg daily medication schedule.
  7. 👤 [USER]: I'm allergic to penicillin, please always remember this.
  8. 🤖 [ASSISTANT]: Important! I've noted your penicillin allergy. This is critical information.
  9. 👤 [USER]: Also rem

---

## Part 4 · Procedural Memory - Workflows & How-To Knowledge

Procedural memory stores **step-by-step workflows** that guide the agent through tasks.

- **How it works**: Procedures are stored with pgvector embeddings. When a user query
  matches a procedure, the agent retrieves and follows the steps.
- **Use case**: "Book an appointment" → retrieves `book_appointment` procedure → follows 7 steps

This makes the agent **consistent** (always follows the same steps) and **explainable**.

In [ ]:
#  Query procedural memory 
from memory.prompts import format_procedures

proc_store = ProceduralMemoryStore()

user_query = "I want to book an appointment with a cardiologist"
logger.info(f" Query: '{user_query}'\n")

procedures = proc_store.query_procedures(user_query, top_k=2, threshold=0.3)

if procedures:
    for proc in procedures:
        logger.success(f" Procedure: {proc.name}  (similarity: {proc.similarity:.2f})")
        print(f"   Category: {proc.category}")
        print(f"   Description: {proc.description}")
        print(f"   Steps: {len(proc.steps)}")
        print()

    # Show formatted procedure (as the agent would see it)
    print("=" * 60)
    print("Formatted for Agent Prompt:")
    print("=" * 60)
    print(format_procedures(procedures))
else:
    logger.error(" No matching procedures found.")
    print("   Run: python scripts/seed_procedures.py")

ℹ️ 🔍 Query: 'I want to book an appointment with a cardiologist'

✅ ✅ Procedure: book_new_appointment  (similarity: 0.53)
   Category: booking
   Description: Book a new appointment for a patient with a doctor at a location
   Steps: 8

✅ ✅ Procedure: find_doctor  (similarity: 0.50)
   Category: inquiry
   Description: Help patient find a suitable doctor by specialty or name
   Steps: 5

Formatted for Agent Prompt:

**Procedure 1: book_new_appointment** (booking)
Description: Book a new appointment for a patient with a doctor at a location
When to use: When a patient requests to schedule a new appointment

Steps:
  1. identify_patient: Verify patient identity using phone number or patient ID. If not found, suggest registration.
  2. identify_requirements: Ask for: specialty needed, preferred doctor (if any), preferred location, date/time preferences, reason for visit.
  3. check_availability: Query CRM database for available doctors matching specialty and location within requested timef

---

## Summary - Memory Types Comparison

| Type | What | How Created | Retrieval | Backend |
|------|------|-------------|-----------|---------|
| **Short-Term** | Recent turns | Every chat message | Recency (last N) | Supabase |
| **Semantic** | Distilled facts | LLM extraction | Cosine similarity | Supabase pgvector |
| **Episodic** | Full conversations | End-of-session snapshot | Cosine similarity on summary | Supabase pgvector |
| **Procedural** | Workflows | Admin-seeded | Cosine similarity | Supabase pgvector |

### How They Work Together in the Agent

```
User query → Memory Recall
                ├─ Short-Term: recent conversation turns (continuity)
                ├─ Semantic: distilled facts (personalisation)
                ├─ Episodic: past session summaries (long-term context)
                └─ Procedural: relevant workflows (task guidance)
             → Combined into memory_context string
             → Injected into Router + Synthesiser prompts
```

### Next Notebook
- **03**: Memory Store & Recall - how memory makes answers smarter (with/without comparison)